In [0]:
%pip install openai
dbutils.library.restartPython()

In [0]:
# 0) Imports & Parameters (SAFE after restart)
import datetime, time
from typing import List, Tuple
from openai import AzureOpenAI
from pyspark.sql import functions as F, types as T
from pyspark.sql import Row

# helper bikin/ambil widget text dgn default
def ensure_widget_text(name: str, default: str) -> str:
    try:
        # kalau belum ada, create
        dbutils.widgets.get(name)
    except:
        dbutils.widgets.text(name, default)
    return dbutils.widgets.get(name)

# widget tanggal (UTC); default = hari ini
p_date_str = ensure_widget_text("p_date", "")
if not p_date_str:
    p_date_str = datetime.datetime.utcnow().date().isoformat()

# widget minimal judul per source; default = "1"
p_min_titles_str = ensure_widget_text("p_min_titles", "1")
try:
    p_min_titles = int(p_min_titles_str)
except ValueError:
    p_min_titles = 1  # fallback aman

print(f"Params → p_date={p_date_str}, p_min_titles={p_min_titles}")
p_date=p_date_str
p_min_titles=p_min_titles

# ----------------
# 1) AOAI Client
# ----------------
SCOPE = "kv-newspulse"  # secret scope (Key Vault–backed)

endpoint    = dbutils.secrets.get(SCOPE, "aoai-endpoint1")
api_key     = dbutils.secrets.get(SCOPE, "aoai-subscription-key1")
deployment  = dbutils.secrets.get(SCOPE, "aoai-deployment1")
api_version = "2024-12-01-preview"  # gunakan yang sama seperti di Azure AI Foundry (View code)

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=endpoint
)

# Sanity check (ringkas 1 kalimat)
test = client.chat.completions.create(
    model=deployment,
    messages=[{"role":"user","content":"Tes singkat: balas 'OK'."}],
    max_tokens=5, temperature=0
)
print("AOAI OK:", test.choices[0].message.content.strip())

# ------------------------
# 2) Azure SQL JDBC CONFIG
# ------------------------
# Ganti sesuai environment kamu
JDBC_URL = "jdbc:sqlserver://sql-newspulse-dev.database.windows.net:1433;databaseName=sqldb-newspulse-dev"
SQL_USER = "sqladmin"
SQL_PWD  = "Danone.20091989"   # (sementara hardcoded sesuai pilihanmu)
JDBC_PROPS = {
    "user": SQL_USER,
    "password": SQL_PWD,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

TABLE_STAGING = "dbo.staging_collect_all_fact_article"
TABLE_SUMMARY = "dwh.fact_article_summary"

# -------------------------------
# 3) Load artikel untuk p_date
# -------------------------------
articles_df = (
    spark.read.jdbc(JDBC_URL, TABLE_STAGING, properties=JDBC_PROPS)
         .filter(F.to_date("load_ts") == F.lit(str(p_date)))
         .select("source_name","Url","title","load_ts")
)

print("Articles today:", articles_df.count())
display(articles_df.limit(10))

# ------------------------------------------
# 4) Helper: Summarize batch via Azure OpenAI
# ------------------------------------------
def summarize_batch(source: str, items: List[dict], day_iso: str,
                    max_titles:int=50, max_tokens:int=600, temperature:float=0.3,
                    retries:int=3, backoff:int=2) -> Tuple[str, str]:
    # Batasi judul untuk efisiensi token
    safe = [it for it in items if it.get("title")]
    safe = safe[:max_titles]
    titles_block = "\n".join([f"- {it['title']} ({it.get('Url','')})" for it in safe])

    prompt = f"""
Summarize the main developments on {day_iso} from source: {source}, based on the news titles below.
Write the output in **English**. Return exactly two sections in Markdown:

### Summary
[3–5 concise sentences in English]

### Highlights
- [3–6 short bullet points with facts/names/numbers in English.]

Titles:
{titles_block}
""".strip()

    last_err = None
    for i in range(retries):
        try:
            r = client.chat.completions.create(
                model=deployment,
                messages=[{"role":"user","content":prompt}],
                max_tokens=max_tokens,
                temperature=temperature,
                top_p=1.0,
            )
            text = r.choices[0].message.content.strip()
            parts = text.split("### Highlights")
            summary_md   = parts[0].replace("### Summary","").strip()
            highlights_md = ("### Highlights" + parts[1]).strip() if len(parts) > 1 else ""
            return summary_md, highlights_md
        except Exception as e:
            last_err = e
            time.sleep(backoff * (i+1))
    return ("Summary unavailable due to API error.",
            f"### Highlights\n- Error: {last_err}")
    
# -------------------------------------------------
# 5) Kelompokkan per source & hindari duplikasi
#    (jangan tulis jika sudah ada untuk tanggal tsb)
# -------------------------------------------------

# Ambil existing summary utk tanggal tsb (untuk anti-duplikat)
existing = (
    spark.read.jdbc(JDBC_URL, TABLE_SUMMARY, properties=JDBC_PROPS)
         .filter(F.col("summary_date") == F.to_date(F.lit(p_date)))
         .select("source_name")
         .distinct()
         .toPandas()
)
already = set((existing["source_name"].fillna("")).tolist()) if len(existing)>0 else set()

grouped = (
    articles_df.groupBy("source_name")
               .agg(F.collect_list(F.struct("title","Url")).alias("items"),
                    F.count("*").alias("cnt"))
               .collect()
)

rows = []
for row in grouped:
    src = row["source_name"] or "Unknown"
    cnt = int(row["cnt"])
    if cnt < p_min_titles:
        continue
    if src in already:
        # sudah ada summary utk tanggal ini & source ini
        continue

    items = [{"title": it["title"], "Url": it["Url"]} for it in row["items"] if it["title"]]
    if not items:
        continue

    summary_md, highlights_md = summarize_batch(src, items, p_date)
    rows.append((p_date, src, cnt, summary_md, highlights_md))

print(f"Prepared {len(rows)} summaries (new only)")

# ------------------------------
# 6) Tulis hasil ke tabel DWH
# ------------------------------
if rows:
    schema = T.StructType([
        T.StructField("summary_date",   T.StringType(),  False),
        T.StructField("source_name",    T.StringType(),  True),
        T.StructField("articles_count", T.IntegerType(), False),
        T.StructField("summary_md",     T.StringType(),  False),
        T.StructField("highlights_md",  T.StringType(),  True),
    ])
    df_out = spark.createDataFrame([Row(*x) for x in rows], schema) \
                  .withColumn("summary_date", F.to_date("summary_date")) \
                  .withColumn("created_ts", F.current_timestamp())

    (df_out
        .select("summary_date","source_name","articles_count","summary_md","highlights_md","created_ts")
        .write.mode("append")
        .jdbc(JDBC_URL, TABLE_SUMMARY, properties=JDBC_PROPS))

    print(f"Inserted {df_out.count()} rows into {TABLE_SUMMARY} for {p_date}")
else:
    print("No new summaries to insert.")